# Generate submittion file for 1K
## You can set size of box (nx, ny), strain, chemical potential range

In [1]:
# from graphenetools import gt
import math
import numpy as np
from graphenetools import gt

### sample is template of commands. You can edit options for simulation here

In [2]:
sample = '-T T? -t 0.003130 -M 8 -I szalewicz -X graphenelut3d '\
    '-E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 '\
    '--strain 0.00000 '\
    '-N N? --Lx Lx? --Ly Ly? '\
    '--graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.??_10.0_101_101_1001_ '\
    '--estimator="number particles" --estimator="planar density average rho"'

In [2]:
def T1maker(nx,ny,strainrun,murange,sample, date='Apr01', fill=0.5, step=1.0):
    #cell maker
    strset = ""
    for ncellx, ncelly in zip((nx, ),(ny, )):
        total_n = 1 * ncellx * ncelly
        ratio = [fill,]*len(strainrun)
        for m,i in enumerate(strainrun):
            iso_strain = 1+0.01*i
            a = 1.42*iso_strain

            lx = a*math.sqrt(3)*1 * ncellx
            ly = a*1.5 * ncelly

            for T in (1.0,):
                if i < 10:
                    strset += "0"+str(i)+" :  "
                else:
                    strset += str(i)+" :  "
                strset += "-T " + f'{T:.3f}' + " -N " + f"{int(total_n*ratio[m])}" + " --Lx " + str(lx) + " --Ly " + str(ly) + "\n"
        print('*'*7+'total n =',str(total_n)+'*'*7)
    strset = strset[:-1]
    print(strset)
    
    #line maker
    linelst = []
    linelst = strset.split("\n")
    print('lines:',len(linelst))
    mumin = [-129.4, -126.8, -124.2, -122.6, -121. , -117.7, -114.3, -112. ,
       -109.7, -109. , -108.4, -106.8, -105.2, -102.7, -100.2,  -98.3,
        -96.3,  -96.2,  -96.2,  -95.2,  -94.2,  -92.9,  -91.5,  -90.3,
        -89.2,  -88. ,  -86.8,  -84.8,  -82.8,  -82.4,  -82.1]
    intmumin = [int(n) for n in mumin]

    line_topush = []
    mulst = []
    
    ###  generate mu samples
    for m,i in enumerate(strainrun):
        if murange[0] == 'vacuum' or murange[0] <= intmumin[i]:
            lowermu = intmumin[i]
        else:
            lowermu = murange[0]
        uppermu = murange[1]
        if lowermu >= uppermu:
            continue
        musamplelist = np.arange(lowermu, uppermu, step)
        line_topush += [linelst[m],]*len(musamplelist) #copy lines for # of mu
        mulst += list(musamplelist)
        print(musamplelist[-1],end=' ')
    print('')
    mu_topush = []
    for num in mulst:
        mkline = " -u"
        for i in (0,):
            push = " "+str(float(num+i))
            mkline += push
        mu_topush.append(mkline)
    print('# lines',len(line_topush))
    print('# mu',len(mu_topush))
    
    #command maker
    commlst = []
    for m, l in enumerate(line_topush):
        locT = l.find("-T")
        locN = l.find("-N")
        locLx = l.find("--Lx")
        locLy = l.find("--Ly")
        locst = l.find(":")
        comm = str(sample)
        comm = comm.replace("T?",str(l[locT+3:locN-1]))
        comm = comm.replace("N?",str(l[locN+3:locLx-1]))
        comm = comm.replace("Lx?",str(l[locLx+5:locLy-1]))
        comm = comm.replace("Ly?",str(l[locLy+5:]))
        comm = comm.replace("??",str(l[:locst-1]))
        comm += mu_topush[m]
        commlst.append(comm)

    print('# command',len(commlst))
    print(commlst[0])
    
    #command writter
    filename = f"../data/pimc_T1_{date}_s{strainrun[0]}-{strainrun[-1]}_{int(musamplelist[-1])}.sh"
    commgen = open(filename, "w")
    ct = 0
    commgen.write("""#!/bin/bash

#SBATCH --partition=bluemoon
#SBATCH --nodes=1
#SBATCH --cpus-per-task=1
#SBATCH --ntasks=1
#SBATCH --mem=1GB
#SBATCH --job-name=pimc
#SBATCH --output=out/pimc-%A-%a.log
#SBATCH --time=30:00:00

# change to the directory where you submitted this script
cd ${SLURM_SUBMIT_DIR}

# Check if the out directory exists, if not, create it
if [ ! -d "./out" ]; then
  mkdir out
fi

# Executable section: echoing some Slurm data
echo "Starting sbatch script `basename $0` at:`date`"
echo "Running host:    ${SLURMD_NODENAME}"
echo "Assigned nodes:  ${SLURM_JOB_NODELIST}"
echo "Job ID:          ${SLURM_JOBID}"

# The job array
case ${SLURM_ARRAY_TASK_ID} in
""")
    for line in commlst:

        commgen.write(str(ct)+")\n")
        commgen.write("sleep "+str(ct*2)+"\n")
        commgen.write("./pimc.e "+ "-p "+ str(ct+1)+" " + line + "\n")
        commgen.write(";;"+"\n")
        ct += 1
    #                     print(line[4:])
    commgen.write("""esac
echo \"Finished run at: `date`\"""")
    commgen.close()
    print('file ready:', filename)

### 'ny' must be even number (boundary condition). 
### strain (third argument) should be iterable (tuple), mu rage is two element tuple with (min, max). You can use 'vacuum' for lower boundary. fill (optional) is initial number of particle filling portion (0 to 1)

In [72]:
T1maker(6,8,(0,1,2,3,4,5,6,7,8,9,10,),('vacuum',-108+0.1),sample, fill = 1/3)

*******total n = 48*******
00 :  -T 1.000 -N 16 --Lx 14.757072880486835 --Ly 17.04
01 :  -T 1.000 -N 16 --Lx 14.904643609291702 --Ly 17.2104
02 :  -T 1.000 -N 16 --Lx 15.052214338096569 --Ly 17.3808
03 :  -T 1.000 -N 16 --Lx 15.199785066901438 --Ly 17.551199999999998
04 :  -T 1.000 -N 16 --Lx 15.347355795706306 --Ly 17.7216
05 :  -T 1.000 -N 16 --Lx 15.494926524511174 --Ly 17.892
06 :  -T 1.000 -N 16 --Lx 15.642497253316044 --Ly 18.0624
07 :  -T 1.000 -N 16 --Lx 15.790067982120913 --Ly 18.2328
08 :  -T 1.000 -N 16 --Lx 15.93763871092578 --Ly 18.403200000000002
09 :  -T 1.000 -N 16 --Lx 16.085209439730647 --Ly 18.5736
10 :  -T 1.000 -N 16 --Lx 16.23278016853552 --Ly 18.744
lines: 11
-108.0 -108.0 -108.0 -108.0 -108.0 -108.0 -108.0 -108.0 -108.0 -108.0 -108.0 
# lines 114
# mu 114
# command 114
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 16 --Lx 14.757072880486835 --Ly 17.04 --graphenelut3d_file_prefi

In [5]:
T1maker(6,8,(0,1,2,3,4,5,6,7,8,9,10,),(-107,-101+0.1),sample,date='Apr03', fill = 1/3)

*******total n = 48*******
00 :  -T 1.000 -N 16 --Lx 14.757072880486835 --Ly 17.04
01 :  -T 1.000 -N 16 --Lx 14.904643609291702 --Ly 17.2104
02 :  -T 1.000 -N 16 --Lx 15.052214338096569 --Ly 17.3808
03 :  -T 1.000 -N 16 --Lx 15.199785066901438 --Ly 17.551199999999998
04 :  -T 1.000 -N 16 --Lx 15.347355795706306 --Ly 17.7216
05 :  -T 1.000 -N 16 --Lx 15.494926524511174 --Ly 17.892
06 :  -T 1.000 -N 16 --Lx 15.642497253316044 --Ly 18.0624
07 :  -T 1.000 -N 16 --Lx 15.790067982120913 --Ly 18.2328
08 :  -T 1.000 -N 16 --Lx 15.93763871092578 --Ly 18.403200000000002
09 :  -T 1.000 -N 16 --Lx 16.085209439730647 --Ly 18.5736
10 :  -T 1.000 -N 16 --Lx 16.23278016853552 --Ly 18.744
lines: 11
-101.0 -101.0 -101.0 -101.0 -101.0 -101.0 -101.0 -101.0 -101.0 -101.0 -101.0 
# lines 77
# mu 77
# command 77
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 16 --Lx 14.757072880486835 --Ly 17.04 --graphenelut3d_file_prefix=.

In [68]:
T1maker(8,8,(0,1,2,3,4,5,),(-112,-90+0.1),sample, fill = 7/16)

*******total n = 64*******
00 :  -T 1.000 -N 28 --Lx 19.676097173982445 --Ly 17.04
01 :  -T 1.000 -N 28 --Lx 19.87285814572227 --Ly 17.2104
02 :  -T 1.000 -N 28 --Lx 20.069619117462093 --Ly 17.3808
03 :  -T 1.000 -N 28 --Lx 20.266380089201917 --Ly 17.551199999999998
04 :  -T 1.000 -N 28 --Lx 20.46314106094174 --Ly 17.7216
05 :  -T 1.000 -N 28 --Lx 20.659902032681565 --Ly 17.892
lines: 6
-90.0 -90.0 -90.0 -90.0 -90.0 -90.0 
# lines 138
# mu 138
# command 138
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 28 --Lx 19.676097173982445 --Ly 17.04 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.00_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -112.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr01_s0-5_-90.sh


In [69]:
T1maker(8,8,(6,7,8,9,10,),(-107,-90+0.1),sample, fill = 7/16)

*******total n = 64*******
06 :  -T 1.000 -N 28 --Lx 20.856663004421392 --Ly 18.0624
07 :  -T 1.000 -N 28 --Lx 21.053423976161216 --Ly 18.2328
08 :  -T 1.000 -N 28 --Lx 21.25018494790104 --Ly 18.403200000000002
09 :  -T 1.000 -N 28 --Lx 21.446945919640864 --Ly 18.5736
10 :  -T 1.000 -N 28 --Lx 21.643706891380692 --Ly 18.744
lines: 5
-90.0 -90.0 -90.0 -90.0 -90.0 
# lines 90
# mu 90
# command 90
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 28 --Lx 20.856663004421392 --Ly 18.0624 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.06_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -107.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr01_s6-10_-90.sh


In [9]:
T1maker(7,8,(0,1,2,3,4,5,),(-112,-88+0.1),sample,date='Apr03_1', fill = 7/16)

*******total n = 56*******
00 :  -T 1.000 -N 24 --Lx 17.21658502723464 --Ly 17.04
01 :  -T 1.000 -N 24 --Lx 17.388750877506986 --Ly 17.2104
02 :  -T 1.000 -N 24 --Lx 17.560916727779333 --Ly 17.3808
03 :  -T 1.000 -N 24 --Lx 17.73308257805168 --Ly 17.551199999999998
04 :  -T 1.000 -N 24 --Lx 17.905248428324022 --Ly 17.7216
05 :  -T 1.000 -N 24 --Lx 18.07741427859637 --Ly 17.892
lines: 6
-88.0 -88.0 -88.0 -88.0 -88.0 -88.0 
# lines 150
# mu 150
# command 150
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 24 --Lx 17.21658502723464 --Ly 17.04 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.00_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -112.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr03_1_s0-5_-88.sh


In [10]:
T1maker(8,10,(0,1,2,3,4,5,),(-112,-88+0.1),sample,date='Apr03_2', fill = 7/16)

*******total n = 80*******
00 :  -T 1.000 -N 35 --Lx 19.676097173982445 --Ly 21.299999999999997
01 :  -T 1.000 -N 35 --Lx 19.87285814572227 --Ly 21.512999999999998
02 :  -T 1.000 -N 35 --Lx 20.069619117462093 --Ly 21.726
03 :  -T 1.000 -N 35 --Lx 20.266380089201917 --Ly 21.938999999999997
04 :  -T 1.000 -N 35 --Lx 20.46314106094174 --Ly 22.151999999999997
05 :  -T 1.000 -N 35 --Lx 20.659902032681565 --Ly 22.365
lines: 6
-88.0 -88.0 -88.0 -88.0 -88.0 -88.0 
# lines 150
# mu 150
# command 150
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 35 --Lx 19.676097173982445 --Ly 21.299999999999997 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.00_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -112.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr03_2_s0-5_-88.sh


In [14]:
T1maker(9,10,(0,1,2,3,4,5,),(-87,-30+0.1),sample,date='Apr03_3', fill = 7/16, step=3.0)

*******total n = 90*******
00 :  -T 1.000 -N 39 --Lx 22.13560932073025 --Ly 21.299999999999997
01 :  -T 1.000 -N 39 --Lx 22.35696541393755 --Ly 21.512999999999998
02 :  -T 1.000 -N 39 --Lx 22.578321507144853 --Ly 21.726
03 :  -T 1.000 -N 39 --Lx 22.799677600352155 --Ly 21.938999999999997
04 :  -T 1.000 -N 39 --Lx 23.02103369355946 --Ly 22.151999999999997
05 :  -T 1.000 -N 39 --Lx 23.24238978676676 --Ly 22.365
lines: 6
-30.0 -30.0 -30.0 -30.0 -30.0 -30.0 
# lines 120
# mu 120
# command 120
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 39 --Lx 22.13560932073025 --Ly 21.299999999999997 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.00_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -87.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr03_3_s0-5_-30.sh


In [3]:
sample = '-T T? -t 0.003130 -M 8 -I szalewicz -X graphenelut3d '\
    '-E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 '\
    '--strain 0.00000 '\
    '-N N? --Lx Lx? --Ly Ly? '\
    '--graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.??_10.0_101_101_1001_ '\
    '--estimator="number particles" --estimator="planar density average rho"'

In [19]:
T1maker(8,8,(0,1,2,3,4,),(-112,-85+0.1),sample,date='Apr05', fill = 7/16, step=1.0)

*******total n = 64*******
00 :  -T 1.000 -N 28 --Lx 19.676097173982445 --Ly 17.04
01 :  -T 1.000 -N 28 --Lx 19.87285814572227 --Ly 17.2104
02 :  -T 1.000 -N 28 --Lx 20.069619117462093 --Ly 17.3808
03 :  -T 1.000 -N 28 --Lx 20.266380089201917 --Ly 17.551199999999998
04 :  -T 1.000 -N 28 --Lx 20.46314106094174 --Ly 17.7216
lines: 5
-85.0 -85.0 -85.0 -85.0 -85.0 
# lines 140
# mu 140
# command 140
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 28 --Lx 19.676097173982445 --Ly 17.04 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.00_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -112.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr05_s0-4_-85.sh


In [21]:
T1maker(8,8,(5,6,7,8,9,10,),(-110,-93+0.1),sample,date='Apr05', fill = 7/16, step=1.0)

*******total n = 64*******
05 :  -T 1.000 -N 28 --Lx 20.659902032681565 --Ly 17.892
06 :  -T 1.000 -N 28 --Lx 20.856663004421392 --Ly 18.0624
07 :  -T 1.000 -N 28 --Lx 21.053423976161216 --Ly 18.2328
08 :  -T 1.000 -N 28 --Lx 21.25018494790104 --Ly 18.403200000000002
09 :  -T 1.000 -N 28 --Lx 21.446945919640864 --Ly 18.5736
10 :  -T 1.000 -N 28 --Lx 21.643706891380692 --Ly 18.744
lines: 6
-93.0 -93.0 -93.0 -93.0 -93.0 -93.0 
# lines 108
# mu 108
# command 108
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 28 --Lx 20.659902032681565 --Ly 17.892 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.05_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -110.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr05_s5-10_-93.sh


In [23]:
T1maker(6,8,(5,6,7,8,9,10,),(-94,-85+0.1),sample,date='Apr05', fill = 7/16, step=1.0)

*******total n = 48*******
05 :  -T 1.000 -N 21 --Lx 15.494926524511174 --Ly 17.892
06 :  -T 1.000 -N 21 --Lx 15.642497253316044 --Ly 18.0624
07 :  -T 1.000 -N 21 --Lx 15.790067982120913 --Ly 18.2328
08 :  -T 1.000 -N 21 --Lx 15.93763871092578 --Ly 18.403200000000002
09 :  -T 1.000 -N 21 --Lx 16.085209439730647 --Ly 18.5736
10 :  -T 1.000 -N 21 --Lx 16.23278016853552 --Ly 18.744
lines: 6
-85.0 -85.0 -85.0 -85.0 -85.0 -85.0 
# lines 60
# mu 60
# command 60
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 21 --Lx 15.494926524511174 --Ly 17.892 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.05_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -94.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr05_s5-10_-85.sh


In [6]:
T1maker(6,8,(26,27,28,29,30),(-65,-40+0.1), sample, date='Apr07', fill = 1, step=1.5)

*******total n = 48*******
26 :  -T 1.000 -N 48 --Lx 18.593911829413408 --Ly 21.470399999999998
27 :  -T 1.000 -N 48 --Lx 18.741482558218276 --Ly 21.6408
28 :  -T 1.000 -N 48 --Lx 18.889053287023145 --Ly 21.8112
29 :  -T 1.000 -N 48 --Lx 19.036624015828014 --Ly 21.9816
30 :  -T 1.000 -N 48 --Lx 19.184194744632883 --Ly 22.151999999999997
lines: 5
-41.0 -41.0 -41.0 -41.0 -41.0 
# lines 85
# mu 85
# command 85
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 200000 -S 1200 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 48 --Lx 18.593911829413408 --Ly 21.470399999999998 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.26_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -65.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr07_s26-30_-41.sh


In [6]:
T1maker(9,10,(0,1,2,3,4,5,6,7,8,9,10,),(-83,-40+0.1),sample,date='Apr19', fill = 1/2, step=2.0)

*******total n = 90*******
00 :  -T 1.000 -N 45 --Lx 22.13560932073025 --Ly 21.299999999999997
01 :  -T 1.000 -N 45 --Lx 22.35696541393755 --Ly 21.512999999999998
02 :  -T 1.000 -N 45 --Lx 22.578321507144853 --Ly 21.726
03 :  -T 1.000 -N 45 --Lx 22.799677600352155 --Ly 21.938999999999997
04 :  -T 1.000 -N 45 --Lx 23.02103369355946 --Ly 22.151999999999997
05 :  -T 1.000 -N 45 --Lx 23.24238978676676 --Ly 22.365
06 :  -T 1.000 -N 45 --Lx 23.463745879974066 --Ly 22.578
07 :  -T 1.000 -N 45 --Lx 23.685101973181368 --Ly 22.791
08 :  -T 1.000 -N 45 --Lx 23.90645806638867 --Ly 23.004
09 :  -T 1.000 -N 45 --Lx 24.12781415959597 --Ly 23.217
10 :  -T 1.000 -N 45 --Lx 24.34917025280328 --Ly 23.43
lines: 11
-41.0 -41.0 -41.0 -41.0 -41.0 -41.0 -41.0 -41.0 -41.0 -41.0 -41.0 
# lines 242
# mu 242
# command 242
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 45 --Lx 22.13560932073025 --Ly 21.299999999999997 --graphenelu

In [12]:
T1maker(9,10,(11,12,13,14,15,),('vacuum',-40+0.1),sample,date='Apr23', fill = 1/2, step=2.0)

*******total n = 90*******
11 :  -T 1.000 -N 45 --Lx 24.57052634601058 --Ly 23.643
12 :  -T 1.000 -N 45 --Lx 24.791882439217883 --Ly 23.856
13 :  -T 1.000 -N 45 --Lx 25.01323853242518 --Ly 24.069
14 :  -T 1.000 -N 45 --Lx 25.234594625632486 --Ly 24.282
15 :  -T 1.000 -N 45 --Lx 25.455950718839784 --Ly 24.494999999999997
lines: 5
-40.0 -41.0 -40.0 -40.0 -40.0 
# lines 160
# mu 160
# command 160
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 45 --Lx 24.57052634601058 --Ly 23.643 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.11_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -106.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr23_s11-15_-40.sh


In [13]:
T1maker(9,10,(16,17,18,19,20,),('vacuum',-40+0.1),sample,date='Apr23', fill = 1/2, step=2.0)

*******total n = 90*******
16 :  -T 1.000 -N 45 --Lx 25.677306812047085 --Ly 24.708
17 :  -T 1.000 -N 45 --Lx 25.898662905254387 --Ly 24.921
18 :  -T 1.000 -N 45 --Lx 26.12001899846169 --Ly 25.134
19 :  -T 1.000 -N 45 --Lx 26.341375091668997 --Ly 25.346999999999994
20 :  -T 1.000 -N 45 --Lx 26.5627311848763 --Ly 25.560000000000002
lines: 5
-40.0 -40.0 -40.0 -41.0 -40.0 
# lines 143
# mu 143
# command 143
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 45 --Lx 25.677306812047085 --Ly 24.708 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.16_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -96.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr23_s16-20_-40.sh


In [6]:
T1maker(9,10,(21,22,23,24,25,26,27),('vacuum',-40+0.1),sample,date='Apr25', fill = 1/2, step=2.0)

*******total n = 90*******
21 :  -T 1.000 -N 45 --Lx 26.784087278083604 --Ly 25.773000000000003
22 :  -T 1.000 -N 45 --Lx 27.005443371290905 --Ly 25.985999999999997
23 :  -T 1.000 -N 45 --Lx 27.226799464498207 --Ly 26.198999999999998
24 :  -T 1.000 -N 45 --Lx 27.448155557705512 --Ly 26.412
25 :  -T 1.000 -N 45 --Lx 27.669511650912813 --Ly 26.624999999999996
26 :  -T 1.000 -N 45 --Lx 27.890867744120115 --Ly 26.837999999999997
27 :  -T 1.000 -N 45 --Lx 28.112223837327416 --Ly 27.051
lines: 7
-40.0 -41.0 -40.0 -41.0 -40.0 -40.0 -40.0 
# lines 176
# mu 176
# command 176
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 45 --Lx 26.784087278083604 --Ly 25.773000000000003 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.21_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -92.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr25_s21-27_-40.sh


In [7]:
T1maker(9,10,(28,29,30,),('vacuum',-40+0.1),sample,date='Apr25', fill = 1/2, step=1.0)

*******total n = 90*******
28 :  -T 1.000 -N 45 --Lx 28.333579930534718 --Ly 27.264
29 :  -T 1.000 -N 45 --Lx 28.55493602374202 --Ly 27.477
30 :  -T 1.000 -N 45 --Lx 28.77629211694932 --Ly 27.689999999999998
lines: 3
-40.0 -40.0 -40.0 
# lines 129
# mu 129
# command 129
-T 1.000 -t 0.003130 -M 8 -I szalewicz -X graphenelut3d -E 300000 -S 2500 -l 8 --relax --bin_size=250 --Lz 10.0 --strain 0.00000 -N 45 --Lx 28.333579930534718 --Ly 27.264 --graphenelut3d_file_prefix=../../pimclocal/LUT/graphene_isotropic_0.28_10.0_101_101_1001_ --estimator="number particles" --estimator="planar density average rho" -u -82.0
file ready: /home/sang/UVM/He4T1set/pimc_T1_Apr25_s28-30_-40.sh
